In [ ]:
%pip install matplotlib kagglehub pandas openpyxl
%pip uninstall -y numpy
%pip install "numpy>=1.23.5,<2"
!python -m pip install protobuf==3.19.6 ml-dtypes==0.2.0
!python -m pip install tensorflow==2.10.0
!python -m pip list | findstr tensorflow

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("apithm/ecg-and-eeg-stress-features")

print("Path to dataset files:", path)

In [ ]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices("GPU"))


In [ ]:
import os
import pandas as pd
ecg_path = os.path.join(path,"ECG (EO, AC1, AC2).xlsx")
ecg_eo = pd.read_excel(ecg_path, sheet_name="EO")
ecg_AC1 = pd.read_excel(ecg_path,sheet_name=("AC1"))
ecg_AC2 = pd.read_excel(ecg_path,sheet_name=("AC2"))

In [ ]:
#label the dataset
ecg_eo["label"] = 0
ecg_AC1["label"] = 1
ecg_AC2["label"] = 2

In [ ]:
#merge the 3 dataframes vertically to create a BIG dataset
ecg_df = pd.concat([ecg_eo,ecg_AC1,ecg_AC2],ignore_index=True)

In [ ]:
#removing columns that willl do data leakage
ecg_df = ecg_df.drop(columns=["Subject NO."])